<a href="https://colab.research.google.com/github/t0brig01/CSE696FinalProject/blob/main/MoralNeuralNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
import pandas as pd
from pandas import concat
from sklearn import preprocessing
print ("TensorFlow version: " + tf.__version__)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE as smt
from sklearn.model_selection import train_test_split



TensorFlow 1.x selected.
TensorFlow version: 1.15.2


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# **Data Loading/Manipulation**

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/t0brig01/CSE696FinalProject/main/TrolleyData.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230475 entries, 0 to 230474
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   _id             230475 non-null  object
 1   Scenario        230475 non-null  object
 2   UserID          230475 non-null  object
 3   Outcome         230475 non-null  int64 
 4   Session_id      230475 non-null  object
 5   Scenario_order  230475 non-null  int64 
 6   Template        230475 non-null  object
 7   answerLeft      230475 non-null  bool  
 8   lang            230475 non-null  object
 9   seenOther       230475 non-null  bool  
 10  country_code    229259 non-null  object
 11  country_full    229259 non-null  object
dtypes: bool(2), int64(2), object(8)
memory usage: 18.0+ MB


In [5]:
df.head()

,_id,Scenario,UserID,Outcome,Session_id,Scenario_order,Template,answerLeft,lang,seenOther,country_code,country_full
0,C3RGCjGPftjMKYHy2,Loop,0000dc12_9518522259818270,1,367585191-9.51852225982e+15,1,Desktop,False,de,True,DE,Germany
1,3F9SfDyofskDPna8R,Footbridge,0000dc12_9518522259818270,0,367585191-9.51852225982e+15,2,Desktop,False,de,True,DE,Germany
2,oz9anPbE6e52TKuJy,Switch,0000dc12_9518522259818270,1,367585191-9.51852225982e+15,3,Desktop,False,de,True,DE,Germany
3,dkAhBaLhzi62Pp6Hp,Footbridge,0002ae2d_2286850331484848,0,1654499857-2.28685033148e+15,1,Desktop,False,en,True,GB,United Kingdom
4,2ussgtw7vCZMNjxSJ,Switch,0002ae2d_2286850331484848,1,1654499857-2.28685033148e+15,2,Desktop,False,en,True,GB,United Kingdom


In [6]:
filtered = df.drop(['Session_id','Template','country_full','_id','answerLeft'], axis=1)
filtered.dropna()
filtered.replace('Loop',0,inplace=True)   #0 is loop trolley 
filtered.replace('Footbridge',1,inplace=True) #1 is footbridge trolley
filtered.replace('Switch',2,inplace=True) #2 is standard trolley

lang = dict()
country = dict()
iLang = 0
iCountry = 0
for index, x in filtered.iterrows():
  if x["lang"] not in lang.keys():
    lang[x["lang"]] = iLang
    iLang = iLang + 1
  if x["country_code"] not in country.keys():
    country[x["country_code"]] = iCountry
    iCountry = iCountry + 1
for key, value in country.items():
  filtered.replace(key,int(value),inplace=True)
for key, value in lang.items():
  filtered.replace(key,int(value),inplace=True)

In [7]:
filtered['country_code'] = filtered['country_code'].astype(int)

In [8]:
filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230475 entries, 0 to 230474
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Scenario        230475 non-null  int64 
 1   UserID          230475 non-null  object
 2   Outcome         230475 non-null  int64 
 3   Scenario_order  230475 non-null  int64 
 4   lang            230475 non-null  int64 
 5   seenOther       230475 non-null  bool  
 6   country_code    230475 non-null  int64 
dtypes: bool(1), int64(5), object(1)
memory usage: 10.8+ MB


In [9]:
filtered.describe()

,Scenario,Outcome,Scenario_order,lang,country_code
count,230475.000000,230475.000000,230475.000000,230475.000000,230475.000000
mean,0.997896,0.700453,1.955301,1.903946,15.234290
std,0.816945,0.458060,0.817113,1.716888,22.201642
min,0.000000,0.000000,1.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,1.000000,3.000000
50%,1.000000,1.000000,2.000000,1.000000,6.000000
75%,2.000000,1.000000,3.000000,3.000000,21.000000
max,2.000000,1.000000,3.000000,9.000000,169.000000


In [10]:
def create_dataset(X):
  Xs = X.drop(["UserID","Outcome"],axis = 1)
  Ys = X.drop(["Scenario","UserID","Scenario_order","lang","seenOther","country_code"],axis=1)
  Xs,Ys = Xs.astype(int),Ys.astype(int)
  Xs['seenOther'] = Xs['seenOther'].astype(bool)
  Ys['Outcome'] = Ys['Outcome'].astype(bool)
  return Xs,Ys

In [11]:
interval = int(len(filtered.index)/3)
x_train, y_train = create_dataset(filtered.head(interval))
x_test, y_test = create_dataset(filtered.tail(interval))
x_val, y_val = create_dataset(filtered.loc[interval:interval*2])

In [103]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76825 entries, 0 to 76824
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Scenario        76825 non-null  int64
 1   Scenario_order  76825 non-null  int64
 2   lang            76825 non-null  int64
 3   seenOther       76825 non-null  bool 
 4   country_code    76825 non-null  int64
dtypes: bool(1), int64(4)
memory usage: 2.4 MB


In [105]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76825 entries, 0 to 76824
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Outcome  76825 non-null  bool 
dtypes: bool(1)
memory usage: 75.1 KB


# **Neural Network**

In [12]:
from keras.models import Sequential
from keras import layers
from keras.layers import LSTM, Dense,Conv1D,MaxPooling1D,Flatten
from keras.optimizers import RMSprop,SGD
from tensorflow import keras
from keras import optimizers
import tensorflow as tf

In [107]:
def create_model():
  model = Sequential()
  model.add(Dense(100, input_dim=5, activation='relu'))
  model.add(Dense(50, activation='sigmoid'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
  return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=2, shuffle=True)
results = cross_val_score(pipeline, x_train, y_train, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/10
38412/38412 [==============================] - 1s 37us/step - loss: 0.6212 - accuracy: 0.6817
Epoch 2/10
38412/38412 [==============================] - 1s 27us/step - loss: 0.6093 - accuracy: 0.7006
Epoch 3/10
38412/38412 [==============================] - 1s 25us/step - loss: 0.6087 - accuracy: 0.7006
Epoch 4/10
38412/38412 [==============================] - 1s 27us/step - loss: 0.6081 - accuracy: 0.7006
Epoch 5/10
38412/38412 [==============================] - 1s 29us/step - loss: 0.6075 - accuracy: 0.7006
Epoch 6/10
38412/38412 [==============================] - 1s 30us/step - loss: 0.6070 - accuracy: 0.7006
Epoch 7/10
38412/38412 [==============================] - 1s 27us/step - loss: 0.6065 - accuracy: 0.7006
Epoch 8/10
38412/38412 [==============================] - 1s 27us/step - loss: 0.6060 - accuracy: 0.7006
Epoch 9/10
38412/38412 [==============================] - 1s 29us/step - loss: 0.6055 - accuracy: 0.7006
Epoch 10/10
38413/38413 [==============================

In [108]:
#Basic
model = Sequential()
model.add(Dense(60, input_dim=5, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(10, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(x_train,y_train,epochs=10, validation_data=(x_val,y_val),batch_size=50)
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print("Results: %.2f%% (%.2f%%)" % (acc*100, loss*100))

Train on 76825 samples, validate on 76826 samples
Epoch 1/10
76825/76825 [==============================] - 8s 100us/step - loss: 0.5949 - accuracy: 0.7006 - val_loss: 0.5873 - val_accuracy: 0.6981
Epoch 2/10
76825/76825 [==============================] - 7s 87us/step - loss: 0.5806 - accuracy: 0.7015 - val_loss: 0.5787 - val_accuracy: 0.7006
Epoch 3/10
76825/76825 [==============================] - 7s 91us/step - loss: 0.5788 - accuracy: 0.7013 - val_loss: 0.5789 - val_accuracy: 0.7010
Epoch 4/10
76825/76825 [==============================] - 7s 91us/step - loss: 0.5780 - accuracy: 0.7020 - val_loss: 0.5818 - val_accuracy: 0.6989
Epoch 5/10
76825/76825 [==============================] - 7s 91us/step - loss: 0.5779 - accuracy: 0.7020 - val_loss: 0.5846 - val_accuracy: 0.6992
Epoch 6/10
76825/76825 [==============================] - 7s 92us/step - loss: 0.5773 - accuracy: 0.7019 - val_loss: 0.5793 - val_accuracy: 0.7007
Epoch 7/10
76825/76825 [==============================] - 7s 96us/s

In [113]:
model = Sequential()
model.add(Dense(12, input_dim=5, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(x_train, y_train, epochs=50, batch_size=25)
# evaluate the keras model
_, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/50
76825/76825 [==============================] - 9s 118us/step - loss: 0.6091 - accuracy: 0.6966
Epoch 2/50
76825/76825 [==============================] - 8s 110us/step - loss: 0.5882 - accuracy: 0.6995
Epoch 3/50
76825/76825 [==============================] - 9s 120us/step - loss: 0.5840 - accuracy: 0.6996
Epoch 4/50
76825/76825 [==============================] - 8s 103us/step - loss: 0.5825 - accuracy: 0.7001
Epoch 5/50
76825/76825 [==============================] - 8s 101us/step - loss: 0.5812 - accuracy: 0.7000
Epoch 6/50
34875/76825 [============>.................] - ETA: 4s - loss: 0.5812 - accuracy: 0.7007

KeyboardInterrupt: ignored

# Epochs vs Batch Experiment

In [13]:
def buildModel():
  model = Sequential()
  model.add(Dense(60, input_dim=5, activation='relu'))
  model.add(Dense(40, activation='relu'))
  model.add(Dense(10, activation='sigmoid'))
  model.add(Dense(1, activation='sigmoid'))
  return model

In [ ]:
acc = dict()
for batch in range(1,15): #batch size 
  for epoch in range(1,15): #epochs
    print("Epochs: {0} \t Batch Size: {1}".format(10*epoch,16*batch))
    model = buildModel()
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=(10*epoch), batch_size=(16*batch),verbose=0)
    loss, accuracy = model.evaluate(x_test, y_test,verbose=0)
    key = "epochs{0}batch{1}".format(10*epoch,16*batch)
    acc[key] = accuracy*100
    print('Accuracy: %.2f%%' % (accuracy*100))

Epochs: 10 	 Batch Size: 16
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Accuracy: 70.39%
Epochs: 20 	 Batch Size: 16
Accuracy: 70.27%
Epochs: 30 	 Batch Size: 16
Accuracy: 70.59%
Epochs: 40 	 Batch Size: 16
Accuracy: 70.60%
Epochs: 50 	 Batch Size: 16
Accuracy: 70.44%
Epochs: 60 	 Batch Size: 16
Accuracy: 70.70%
Epochs: 70 	 Batch Size: 16
Accuracy: 70.38%
Epochs: 80 	 Batch Size: 16
Accuracy: 70.79%
Epochs: 90 	 Batch Size: 16
Accuracy: 70.85%
Epochs: 100 	 Batch Size: 16
Accuracy: 70.80%
Epochs: 110 	 Batch Size: 16
Accuracy: 70.66%
Epochs: 120 	 Batch Size: 16
Accuracy: 70.85%
Epochs: 130 	 Batch Size: 16
Accuracy: 70.72%
Epochs: 140 	 Batch Size: 16
Accuracy: 70.70%
Epochs: 10 	 Batch Size: 32
Accuracy: 70.41%
Epochs: 20 	 Batch Size: 32
Accuracy: 70.28%
Epochs: 30 	 Batch Size: 32
Accuracy: 70.56%
Epochs: 40 	 Batch Size: 32
Accuracy: 70.51%
Epochs: 5